In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from datetime import datetime

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

pio.renderers.default = "notebook"
pio.templates.default = "plotly_white"


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

In [2]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
periodo_historico = cfg.BASELINE
periodo_de_estudio = cfg.STUDY

In [4]:
def show_response_contents(df):
    print("The response contains:")
    print(json.dumps(list(df['variable'].unique()), sort_keys=True, indent=4))
    print(json.dumps(list(df['device'].unique()), sort_keys=True, indent=4))

In [5]:
df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)

In [6]:
df = df.sort_values(by=['variable','datetime'])
df = pro.datetime_attributes(df)

In [7]:
df_chiller = df.copy()
df_chiller = df_chiller[df_chiller['variable'].isin(['eficiencia-chiller','factor-de-potencia','potencia-termica','potencia-activa','factor-carga-chiller'])]

In [8]:
df_chiller = df_chiller[df_chiller['device'].isin(['cvl---chiller'])]
#df_chiller_2 = df_chiller.copy()

### Rendimientos Chiller

In [9]:
df_chiller

,value,variable,device,device_name,hour,day,dow,cont_dow,week,month,year
datetime,,,,,,,,,,,
2023-08-01 00:00:00-05:00,0.770850,eficiencia-chiller,cvl---chiller,CVL - Chiller,0,1,martes,1.000000,31,8,2023
2023-08-01 00:05:00-05:00,0.571597,eficiencia-chiller,cvl---chiller,CVL - Chiller,0,1,martes,1.000000,31,8,2023
2023-08-01 00:10:00-05:00,0.806612,eficiencia-chiller,cvl---chiller,CVL - Chiller,0,1,martes,1.000000,31,8,2023
2023-08-01 00:15:00-05:00,0.746978,eficiencia-chiller,cvl---chiller,CVL - Chiller,0,1,martes,1.000000,31,8,2023
2023-08-01 00:20:00-05:00,0.861085,eficiencia-chiller,cvl---chiller,CVL - Chiller,0,1,martes,1.000000,31,8,2023
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-30 23:40:00-05:00,165.750585,potencia-termica,cvl---chiller,CVL - Chiller,23,30,sábado,5.958333,13,3,2024
2024-03-30 23:45:00-05:00,162.483750,potencia-termica,cvl---chiller,CVL - Chiller,23,30,sábado,5.958333,13,3,2024
2024-03-30 23:50:00-05:00,168.810972,potencia-termica,cvl---chiller,CVL - Chiller,23,30,sábado,5.958333,13,3,2024


In [10]:
df_pivot

NameError: name 'df_pivot' is not defined

In [ ]:
#df_chiller = df_chiller.groupby(['variable','hour'])['value'].mean().reset_index()

# Agrupar por 'hour' y 'variable', y calcular la media de 'value'
#df_grouped = df_chiller.groupby(['hour', 'variable']).median().reset_index()



# Pivotar el DataFrame para tener 'potencia-activa' y 'potencia-termica' como columnas
df_pivot = df_chiller.pivot(index='hour', columns='variable', values='value')

In [ ]:
df_chiller = df_chiller.drop_duplicates(subset=['hour', 'variable'])
df_pivot = df_chiller.pivot(index='hour', columns='variable', values='value')


In [ ]:
# Crear un gráfico de dispersión
figx = px.scatter(df_pivot, x='potencia-activa', y='eficiencia-chiller', trendline='ols')

# Actualizar el título y las etiquetas
figx.update_layout(title='Relación entre Potencia Activa y Eficiencia Chiller',  xaxis_title='Potencia Activa', yaxis_title='Eficiencia Chiller')

# Mostrar el gráfico
figx.show()

In [ ]:
# Agrupar por 'hour' y 'variable', y calcular la media de 'value'
#df_grouped = df_chiller.groupby(['hour', 'variable']).mean().reset_index()

# Pivotar el DataFrame para tener 'potencia-activa' y 'potencia-termica' como columnas
#df_pivot = df_grouped.pivot(index='hour', columns='variable', values='value')

# Crear un gráfico de dispersión
fig = px.scatter(df_pivot, x='potencia-termica', y='potencia-activa', trendline='ols')

# Actualizar el título y las etiquetas
fig.update_layout(title='Relación entre Potencia Activa y Potencia Termica', xaxis_title='Potencia Termica', yaxis_title='Potencia Activa')

# Mostrar el gráfico
fig.show()

In [ ]:
# Crear un gráfico de dispersión
fig1 = px.scatter(df_pivot, x='factor-carga-chiller', y='eficiencia-chiller', trendline='ols')

# Actualizar el título y las etiquetas
fig1.update_layout(title='Relación entre Factor de carga chiller  y Eficiencia Chiller', xaxis_title='Factor de carga chiller', yaxis_title='Eficiencia Chiller')

# Mostrar el gráfico
fig1.show()